In [ ]:
# taken from: https://www.sbert.net/examples/unsupervised_learning/SimCSE/README.html

In [1]:
from sentence_transformers import SentenceTransformer, InputExample
from sentence_transformers import models, losses
from torch.utils.data import DataLoader
import torch
import pandas as pd
import sqlite3
model_path = '../models/'

In [2]:
con_rev_en_sent = sqlite3.connect('../finetuing_datasets/review_en_train.db')
df_rev_en_sent = pd.read_sql_query("SELECT note FROM review_en_train", con_rev_en_sent)
df_rev_en_sent.head(n=2)

,note
0,very smooth port
1,complex but simply like


In [2]:
# best from: https://www.sbert.net/docs/pretrained_models.html#semantic-search
model_name = 'sentence-transformers/all-MiniLM-L6-v2'
word_embedding_model = models.Transformer(model_name, max_seq_length=128)
pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension())
model = SentenceTransformer(modules=[word_embedding_model, pooling_model])

In [12]:
train_sentences = df_rev_en_sent.note.to_list()

train_data = [InputExample(texts=[s, s]) for s in train_sentences]

train_dataloader = DataLoader(train_data, batch_size=32)

train_loss = losses.MultipleNegativesRankingLoss(model)

In [13]:
mps_device = torch.device("mps")
model = model.to(mps_device)

In [14]:
model.fit(
    train_objectives=[(train_dataloader, train_loss)],
    epochs=1,
    show_progress_bar=True
)

# model.save(model_path + 'simcse_en')

Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/59850 [00:00<?, ?it/s]

KeyboardInterrupt: 